# Save and Load a model pipeline

In [ ]:
from river import metrics, datasets, compose, preprocessing
from river_torch.classification import Classifier
from torch import nn

In [ ]:
dataset = datasets.Phishing()
metric = metrics.Accuracy()

class MyModule(nn.Module):
    def __init__(self, n_features):
        super(MyModule, self).__init__()
        self.dense0 = nn.Linear(n_features,5)
        self.nonlin = nn.ReLU()
        self.dense1 = nn.Linear(5, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.nonlin(self.dense1(X))
        X = self.softmax(X)
        return X

model_pipeline = compose.Pipeline(
    preprocessing.StandardScaler,
    Classifier(module=MyModule,loss_fn="binary_cross_entropy",optimizer_fn='adam')
)
model_pipeline

## Run the model

In [ ]:
i = 0
for x,y in dataset:
    if i == 1000:
        break
    y_pred = model_pipeline.predict_one(x)      # make a prediction
    metric = metric.update(y, y_pred)  # update the metric
    model_pipeline = model_pipeline.learn_one(x, y)    # make the model learn
    i += 1
print(f'Accuracy: {metric.get()}')

## Save the model pipeline

In [ ]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model_pipeline,f)

## Load the model pipeline

In [ ]:
with open('model.pkl', 'rb') as f:
    model_pipeline = pickle.load(f)

In [ ]:
for x,y in dataset:
    y_pred = model_pipeline.predict_one(x)      # make a prediction
    metric = metric.update(y, y_pred)  # update the metric
    model_pipeline = model_pipeline.learn_one(x, y)    # make the model learn
    i += 1
print(f'Accuracy: {metric.get()}')